Create a Kafka producer client using the given config, and produce 100 messages using `faker` of 
```
{
    'id': uuid4(),
    'name': name(),
    'email': email(),
    'timestamp': time.time()
}
```
to topic called `msds682.day3`

In [ ]:
import json
import os
import time

from confluent_kafka import Producer
from dotenv import load_dotenv
from faker import Faker

In [ ]:
load_dotenv()

In [ ]:
conf = {
        'bootstrap.servers': os.environ['CONFLUENT_SERVER'],
        'security.protocol': 'SASL_SSL',
        'sasl.mechanism': 'PLAIN',
        'sasl.username': os.environ['CONFLUENT_API_KEY'],
        'sasl.password': os.environ['CONFLUENT_API_SECRET'],
    }

In [ ]:
# Create the producer instance
producer = Producer(conf)

# Initialize data generator
fake = Faker()

In [ ]:
for i in range(100):
    # Generate mock user data
    user_data = {
        'id': fake.uuid4(),
        'name': fake.name(),
        'email': fake.email(),
        'timestamp': time.time()
    }
    
    # Serialize the data to a JSON string
    json_data = json.dumps(user_data)
    
    # Produce the message to the Kafka topic
    # `on_delivery` specifies the callback function.
    producer.produce(
        topic='msds682.day3',
        key=str(user_data['id']),
        value=json_data.encode('utf-8'),
    )
    
    # Flush any outstanding or buffered messages to the Kafka broker.
    producer.poll(0)
    
    # Pause for a moment to simulate a real-time stream
    time.sleep(1)